## Educational analytic case with McDonalds and Starbucks menu: what would be better as meals for teens?

[Case description] Restrictions from the task description [Restrictions, document-1 = link, document-2 = link]

DONE:
- values from norms (document-1) extracted manually --> txt --> dict. We find only 6 values: Energy(kcal), Fat(g), Carbohydrates(g), Fiber(g), Protein(g), Sodium (g), wich exist in all three datasets (document-1, McD and SB menus)
- values from McD and SB .csv menus --> Excel --> (processing: combine, add features in cols, enrichment)
- features added: (1) 'McD=0_SB=1': restraunt coding, (2) 'Category': food/drink, (3) 'Kind': bakery/breakfast/hot drink etc.
- dataset enriched manually: about 60 items in 'Snacks & sweets' category were added for SB menu. Thus we have more balanced dataaset (McD/SB share).
- discard from zeroes: dividing all mass values by 1000 (mg --> g)
- several bug fixes in values
- deleted items with all zeroes
- calculate weighted norms - according the rule requirement from official document-2
- deleting dishes with > 60% (breakfast + dinner) of day norm caloric or nutrition values (for instance, 1880 kcal (40 pieces of chicken McNuggets)

PLAN:
- include potential allergics - new feature?
- add the data on serving sizes
- search for nutrition indices or calculate integrated nutrition index (may be Carb:Fat:Prot matching those in norms?) and visualize it
- develop the decision making algorithm for combining potential breakfast and dinner sets 
- test the algorithm, corrections
- extract the results, preference for hot dishes and moderate-sugared drinks

In [1]:
# required imports
import pandas as pd
from itertools import product, combinations
import matplotlib
%matplotlib inline

### Official norms

In [2]:
# Dict with actual norms (from official document-1)
norms = {}

f = open ('actual_norms.txt')

for line in f:
    line = line.strip().split(',')
    norms[line[0]] = line[1]
    
del norms['Item']
norms

{'Energy(kcal)': '2900',
 'Fat(g)': '97',
 'Carbohydrates(g)': '421',
 'Fiber(g)': '20',
 'Protein(g)': '87',
 'Sodium(g)': '1.3'}

In [3]:
# (From official document-2): 
# -breakfast + dinner = 20-25% + 30-35% daily energy value respectively --> 
# 25% and 35% (max due to sports competitions)
# -breakfast + dinner = 55-60% total daily nutrients value --> 
# 25% and 35% (max due to sports competitions)
# (only breakfast and dinner mentioned in the task)
# Let's assume that breakfast / dinner share are equal --> 
# meal_norm (30% and 30% respectively)

# calculating weighted norms:

meal_norm = {}

for key, value in norms.items():
    meal_norm[key] = float(value)*0.3
    
# sum_norm dict for data filtering (del items which contain greater values):   
sum_norm = {key:float(value)*0.6 for key, value in norms.items()}
    
print (meal_norm)
print (sum_norm)

{'Energy(kcal)': 870.0, 'Fat(g)': 29.099999999999998, 'Carbohydrates(g)': 126.3, 'Fiber(g)': 6.0, 'Protein(g)': 26.099999999999998, 'Sodium(g)': 0.39}
{'Energy(kcal)': 1740.0, 'Fat(g)': 58.199999999999996, 'Carbohydrates(g)': 252.6, 'Fiber(g)': 12.0, 'Protein(g)': 52.199999999999996, 'Sodium(g)': 0.78}


In [4]:
# Earlier in 'data_processing' notebook we increased the sodium sum 
# (breakfast + dinner) norm from 0.78 to 1.0, i.e. +0.22. 
# In turns it increases sodium meal_norm by 0.11 g. Dicts should be updated: 
meal_norm['Sodium(g)'] += 0.11
sum_norm['Sodium(g)'] += 0.22
print (meal_norm)
print (sum_norm)

{'Energy(kcal)': 870.0, 'Fat(g)': 29.099999999999998, 'Carbohydrates(g)': 126.3, 'Fiber(g)': 6.0, 'Protein(g)': 26.099999999999998, 'Sodium(g)': 0.5}
{'Energy(kcal)': 1740.0, 'Fat(g)': 58.199999999999996, 'Carbohydrates(g)': 252.6, 'Fiber(g)': 12.0, 'Protein(g)': 52.199999999999996, 'Sodium(g)': 1.0}


In [5]:
# df for joining results
results_all = pd.DataFrame()

### McD & SB menus

In [6]:
# Starbucks (SB) and McDonalds (McD) combined and processed menu: 
menu_df = pd.read_csv ('combined_processed.csv', sep=',', encoding = 'koi8-r')
menu_df.head()

McD/SB Category    Kind               Item  Energy(kcal)  Fat(g)  \
0     SB     food  Bakery       Chonga Bagel           300     5.0   
1     SB     food  Bakery       8-Grain Roll           380     6.0   
2     SB     food  Bakery   Almond Croissant           410    22.0   
3     SB     food  Bakery   Banana Nut Bread           420    22.0   
4     SB     food  Bakery  Birthday Cake Pop           170     9.0   

   Carbohydrates(g)  Fiber(g)  Protein(g)  Sodium(g)  
0                50       3.0          12       0.53  
1                70       7.0          10       0.43  
2                45       3.0          10       0.39  
3                52       2.0           6       0.32  
4                23       0.0           1       0.11

### Idea for core algorithm for menu making 
1. Normalize values of all items by dividing them on meal_norm values dict. <b>[DONE]</b>
2. Develop the format of breakfast / dinner (for instance, [1-st food item, 2-nd food item, 3-rd drink item]) <b>[DONE]</b>
3. Build a meal-maker function (dataset, format) which returns a result_dataset of combitations of items from dataset categories specified in format template <b>[DONE]</b>
4. Build an quality assement function, which recieves result_dataset and evaluate caloric and nutrition: 6 total values derived from all items in certain meal combitation and returns 'B-index' (Balance index): how that combination fits the norms. If 'B-index' close to 0 (StDev of six values that normalized as share of 1, noted above in p.1) the meal combination well fitted <b>[DONE]</b>
5. Evaluation of SB VS McD menus by this algorithm and draw the conclusions

In [7]:
# normalizing whole dataset for meal_norm --> new features:
for item in menu_df[['Energy(kcal)','Fat(g)', 'Carbohydrates(g)','Fiber(g)', 'Protein(g)', 'Sodium(g)']]:
    for key in meal_norm:
        if key == item:
            menu_df['n_'+item] = menu_df[item] / float (meal_norm[key])

menu_df.head()
# menu_df.to_csv('menu_normalized.csv')

McD/SB Category    Kind               Item  Energy(kcal)  Fat(g)  \
0     SB     food  Bakery       Chonga Bagel           300     5.0   
1     SB     food  Bakery       8-Grain Roll           380     6.0   
2     SB     food  Bakery   Almond Croissant           410    22.0   
3     SB     food  Bakery   Banana Nut Bread           420    22.0   
4     SB     food  Bakery  Birthday Cake Pop           170     9.0   

   Carbohydrates(g)  Fiber(g)  Protein(g)  Sodium(g)  n_Energy(kcal)  \
0                50       3.0          12       0.53        0.344828   
1                70       7.0          10       0.43        0.436782   
2                45       3.0          10       0.39        0.471264   
3                52       2.0           6       0.32        0.482759   
4                23       0.0           1       0.11        0.195402   

   n_Fat(g)  n_Carbohydrates(g)  n_Fiber(g)  n_Protein(g)  n_Sodium(g)  
0  0.171821            0.395883    0.500000      0.459770         1.06  
1  0.206186            0.554236    1.166667      0.383142         0.86  
2  0.756014            0.356295    0.500000      0.383142         0.78  
3  0.756014            0.411718    0.333333      0.229885         0.64  
4  0.309278            0.182106    0.000000      0.038314         0.22

### Preserve McD menu

In [8]:
McD_menu = menu_df.iloc[217:]
McD_menu.drop(['McD/SB', 'Energy(kcal)', 
               'Fat(g)', 'Carbohydrates(g)', 
               'Fiber(g)', 'Protein(g)', 'Sodium(g)'], axis=1, inplace=True)

McD_menu.reset_index(drop=True, inplace=True)
McD_menu.head ()

C:\Users\u1\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Category           Kind                              Item  n_Energy(kcal)  \
0     food  Hot Breakfast                      Egg McMuffin        0.344828   
1     food  Hot Breakfast                 Egg White Delight        0.287356   
2     food  Hot Breakfast                  Sausage McMuffin        0.425287   
3     food  Hot Breakfast         Sausage McMuffin with Egg        0.517241   
4     food  Hot Breakfast  Sausage McMuffin with Egg Whites        0.459770   

   n_Fat(g)  n_Carbohydrates(g)  n_Fiber(g)  n_Protein(g)  n_Sodium(g)  
0  0.446735            0.245447    0.666667      0.651341         1.50  
1  0.274914            0.237530    0.666667      0.689655         1.54  
2  0.790378            0.229612    0.666667      0.536398         1.56  
3  0.962199            0.237530    0.666667      0.804598         1.72  
4  0.790378            0.237530    0.666667      0.804598         1.76

In [9]:
# all McD item groups
McD_menu['Kind'].unique()

array(['Hot Breakfast', 'Hot Big Breakfast', 'Sandwiches', 'Burgers',
       'Salads', 'Chiken Snacks', 'Sweets/Desserts', 'Sweets/Snacks',
       'Cold Drinks', 'Hot Drinks'], dtype=object)

In [16]:
# McD menu: same assumptions but different item groups:
McD_first = ['Salads', 'Chiken Snacks']
McD_second = ['Hot Big Breakfast', 'Hot Breakfast', 'Sandwiches', 'Burgers']
McD_desserts = ['Sweets/Desserts', 'Sweets/Snacks']
McD_snacks = ['Chiken Snacks', 'Snacks']
McD_drinks = ['Cold Drinks', 'Hot Drinks']

# Lists for combinations. Current list should be passed to
# 'McD_current' variable for unpacking and futher processing (bellow)

# 3-item 'solid breakfast':
McD_b = [McD_second, McD_drinks] 

# 2-item 'light breakfast':
McD_light_b = [McD_first, McD_drinks]

# 3-item 'solid dinner':
McD_d = [McD_snacks, McD_desserts, McD_drinks]

# 2-item 'light dinner':
McD_light_d = [McD_first, McD_snacks, McD_drinks]

# 2-item 'snack breakfast/dinner':
McD_snack = [McD_snacks, McD_drinks]

# all non-repeated elemwnt-wise combinatorial trick with Itertools 'product' function
McD_current = list(product(*McD_light_d))
# McD_current
len (McD_current)

8

In [17]:
# Full set of combinations for 2-item breakfast / 3-item dinner formats from above catrgories
# Not full-automated in cycle and bad code... 

# 3-item or 2-item meals (manual switch):

k1, k2, k3 = [], [], []
# k1, k2 = [], []

def kind_lists (row, args=McD_current[0]): # this should be iterated manually from 0 to (len(..._current) - 1)
    
    '''making lists of all items that matches with respective kind in '..._current' format
    3 (or 2) kind-format --> 3 (or 2) lists of items'''
    
    # Well, not so elegant code... But slices cause kernel death

    if args[0] in row[1]:
        k1.append (list ([row[2], row[3], row[4], row[5], row[6], row[7], row[8]])) 
    if args[1] in row[1]:
        k2.append (list ([row[2], row[3], row[4], row[5], row[6], row[7], row[8]])) 
    if args[2] in row[1]:
        k3.append (list ([row[2], row[3], row[4], row[5], row[6], row[7], row[8]])) 
    
    return k1,k2,k3

McD_menu.apply (kind_lists, axis=1)

# make all non-repeated element-wise combinations of items (by default of 3 items = len (kinds))
arrs = [k1,k2,k3]
combinator = list(product(*arrs))
df_combi = pd.DataFrame (combinator, columns = ['first', 'second', 'third'])
df_combi.head()

first  \
0  [Premium Bacon Ranch Salad (without Chicken), ...   
1  [Premium Bacon Ranch Salad (without Chicken), ...   
2  [Premium Bacon Ranch Salad (without Chicken), ...   
3  [Premium Bacon Ranch Salad (without Chicken), ...   
4  [Premium Bacon Ranch Salad (without Chicken), ...   

                                              second  \
0  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   
1  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   
2  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   
3  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   
4  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   

                                               third  
0  [Sweet Tea (Small), 0.1724137931034483, 0.0, 0...  
1  [Sweet Tea (Medium), 0.20689655172413793, 0.0,...  
2  [Sweet Tea (Large), 0.25287356321839083, 0.0, ...  
3  [Sweet Tea (Child), 0.12643678160919541, 0.0, ...  
4  [Latte (Small), 0.19540229885057472, 0.3092783...

In [18]:
# Let's evaluate all these combinations in terms of total calories and nutritions 
# (Yes it could be designed with more pythonic and elegant code...)

'''df-applied functions calculate inversions of weighted sums. Next is calculating 
    of B-index as mean of weighted (described above) values in row of elements 
    representing a cetrain meal variant. Returns same df featured by 6 inversed
    weighted sums and B_index column. ZeroDivisionError catched --> value setted to 0.
    
    So the more B_index equal to 1, the more balanced items combination we found.
    If 'B-index' close to 1 (mean of six inv. values that normalized (weighted ) 
    as share of 1, noted above in p.1) then meal combination well fitted.'''

def e_sum (row):
    e_sum = row[2][1] + row[3][1] + row[4][1]
    return e_sum
    
def f_sum (row):   
    f_sum = row[2][2] + row[3][2] + row[4][2]
    return f_sum
    
def c_sum (row):       
    c_sum = row[2][3] + row[3][3] + row[4][3]        
    return c_sum
        
def fi_sum (row):
    fi_sum = row[2][4] + row[3][4] + row[4][4]
    return fi_sum
    
def p_sum (row): 
    p_sum = row[2][5] + row[3][5] + row[4][5]
    return p_sum
    
def s_sum (row):
    s_sum = row[2][6] + row[3][6] + row[4][6]
    return s_sum

df_combi.insert(0, 'McD/SB', 'McD_light_dinner')
df_combi.insert(1, 'Kind', str(McD_current[0]))
df_combi['Energy'] = df_combi.apply (e_sum, axis=1)
df_combi['Fat'] = df_combi.apply (f_sum, axis=1)
df_combi['Carbohydrates'] = df_combi.apply (c_sum, axis=1)
df_combi['Fiber'] = df_combi.apply (fi_sum, axis=1)
df_combi['Protein'] = df_combi.apply (p_sum, axis=1)
df_combi['Sodium'] = df_combi.apply (s_sum, axis=1)

In [19]:
# full item combinations from first group combination in terms of total normalized
# caloric and nutrition values. Finally mean and std of these sums:
df_combi['MEAN'] = df_combi.mean(axis=1)
df_combi['STDEV'] = df_combi.iloc[:,:-1].std(axis=1)
df_combi

McD/SB                                        Kind  \
0     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
1     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
2     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
4     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
5     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
6     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
7     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
8     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
9     McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
10    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
11    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
12    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
13    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
14    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
15    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
16    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
17    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
18    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
19    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
20    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
21    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
22    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
23    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
24    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
25    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
26    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
27    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
28    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
29    McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
...                ...                                         ...   
3610  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3611  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3612  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3613  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3614  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3615  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3616  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3617  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3618  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3619  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3620  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3621  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3622  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3623  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3624  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3625  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3626  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3627  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3628  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3629  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3630  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3631  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3632  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3633  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3634  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3635  McD_light_dinner  ('

In [23]:
# results df building
results_all = results_all.append(df_combi)
results_all.head()

McD/SB                                        Kind  \
0  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
1  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
2  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
3  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   
4  McD_light_dinner  ('Salads', 'Chiken Snacks', 'Cold Drinks')   

                                               first  \
0  [Premium Bacon Ranch Salad (without Chicken), ...   
1  [Premium Bacon Ranch Salad (without Chicken), ...   
2  [Premium Bacon Ranch Salad (without Chicken), ...   
3  [Premium Bacon Ranch Salad (without Chicken), ...   
4  [Premium Bacon Ranch Salad (without Chicken), ...   

                                              second  \
0  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   
1  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   
2  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   
3  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   
4  [Chipotle BBQ Snack Wrap (Crispy Chicken), 0.3...   

                                               third    Energy       Fat  \
0  [Coca-Cola Classic (Small), 0.1609195402298850...  0.712644  0.756014   
1  [Coca-Cola Classic (Medium), 0.229885057471264...  0.781609  0.756014   
2  [Coca-Cola Classic (Large), 0.3218390804597701...  0.873563  0.756014   
3  [Coca-Cola Classic (Child), 0.1149425287356321...  0.666667  0.756014   
4  [Diet Coke (Small), 0.0, 0.0, 0.0, 0.0, 0.0, 0...  0.551724  0.756014   

   Carbohydrates     Fiber   Protein  Sodium      MEAN     STDEV  
0       0.680918  0.666667  0.881226    2.16  0.976245  0.585038  
1       0.807601  0.666667  0.881226    2.17  1.010519  0.572310  
2       0.973872  0.666667  0.881226    2.17  1.053557  0.557334  
3       0.593824  0.666667  0.881226    2.16  0.954066  0.598962  
4       0.372130  0.666667  0.881226    2.18  0.901293  0.650209

In [21]:
# # writing results df
# results_all.reset_index(inplace=True, drop=True)
# results_all.to_csv('McD_light_d_results.csv', index=False)